In [1]:
import numpy as np
import pandas as pd
import vtk
from dfninverse import DFNINVERSE
from numpy.random import randint, uniform, normal, random
from numpy.linalg import det, inv, norm
import matplotlib.pyplot as plt
from numpy import diff
from helper import latexify, vtk_show, vtk_meshrender, vtk_interactiveshow
from model import mcmc_sampler
%matplotlib inline

## Display the synthetic DFN Model and field observation

In [2]:
# Set synthetic model information
station_coordinates = [(0.4, 0.4, 0.2),   (0.4, 0.4, -0.2),    (0.4, -0.4, 0.2),
                       (0.4, -0.4, -0.2), (-0.15, -0.08, 0.2), (-0.15, -0.08, 0)]

syndfn_mesh = '/Volumes/SD_Card/Thesis_project/synthetic_model/output/full_mesh.vtk'
syndfn_observation = '/Volumes/SD_Card/Thesis_project/synthetic_model/output/obs_readings.csv'

In [3]:
# Uncomment this cell to show the synthetic model
# mesh_render = vtk_meshrender(syndfn_mesh, station_coordinates)
# vtk_show(mesh_render, 600, 600)

In [4]:
# ncomment this cell to plot field observation
# df = pd.read_csv(syndfn_observation, index_col=0)
# fig = plt.figure(figsize=[8, 6])
# for col in df.columns:
#     plt.plot(np.arange(0, len(df)), df[col], label=col)
# plt.xlabel('time step')
# plt.ylabel('Pressure (Pa)')
# plt.legend()
# plt.title('Field Observation of the Synthetic DFN Model')
# plt.show()

# Run Inversion

In [5]:
# Set inverse project information
project_path = '/Volumes/SD_Card/Thesis_project/dfn_test'
field_observation_file = '/Volumes/SD_Card/Thesis_project/synthetic_model/output/obs_readings.csv'
ncpu = 1
dfninv = DFNINVERSE(project_path, field_observation_file, station_coordinates, ncpu)
field_observation = dfninv.obs_data

In [6]:
# Define the initial fractures
observed_fractures = np.asarray([[-0.4, 0, 0, 0, np.pi / 2, 0.85],
                                 [0.3, 0, 0.2, 0, np.pi / 2, 0.7],
                                 [0.4, 0, -0.2, 0, np.pi / 2, 0.8]])
inferred_fractures = np.asarray([[-0.15, 0, 0.2, 0, 0, 0.8]])

n_inferred_frac = inferred_fractures.shape[0]
n_observed_frac = observed_fractures.shape[0]

# Define covariance matrix \Sigma_f, \Sigma_v
sig_obs = 0.0001
sig_unknown = 1

sig_f = np.hstack((sig_obs * np.ones(n_observed_frac), sig_unknown * np.ones(n_inferred_frac)))
sig_v = np.asarray([0.1, 1e-3, 1e-3, 1e-3, 1e-3, 1e-3])

In [7]:
mcmc = mcmc_sampler(dfninv, field_observation, var_sigma=[sig_f, sig_v], moves='D')

In [8]:
s_initial = np.vstack((observed_fractures, inferred_fractures))

In [9]:
chain = mcmc.sample(s_initial, 10)

/Volumes/SD_Card/Thesis_project/pydfninv/model.py:195: RuntimeWarning: invalid value encountered in double_scalars
  prior_ratio = prior_1 / prior_0


******************************Iteration:0******************************
Move:D, Prior=0.0, RMS=337.00255105470836
Accept: True


/Volumes/SD_Card/Thesis_project/pydfninv/model.py:195: RuntimeWarning: invalid value encountered in double_scalars
  prior_ratio = prior_1 / prior_0
/Volumes/SD_Card/Thesis_project/pydfninv/model.py:195: RuntimeWarning: invalid value encountered in double_scalars
  prior_ratio = prior_1 / prior_0


******************************Iteration:1******************************
Move:D, Prior=0.0, RMS=337.00255105470836
Accept: True
******************************Iteration:2******************************
Move:D, Prior=0.0, RMS=337.00255105470836
Accept: True
******************************Iteration:3******************************
Move:D, Prior=0.0, RMS=337.00255105470836
Accept: True


/Volumes/SD_Card/Thesis_project/pydfninv/model.py:195: RuntimeWarning: invalid value encountered in double_scalars
  prior_ratio = prior_1 / prior_0
/Volumes/SD_Card/Thesis_project/pydfninv/model.py:195: RuntimeWarning: invalid value encountered in double_scalars
  prior_ratio = prior_1 / prior_0
/Volumes/SD_Card/Thesis_project/pydfninv/model.py:195: RuntimeWarning: invalid value encountered in double_scalars
  prior_ratio = prior_1 / prior_0


...

/Volumes/SD_Card/Thesis_project/pydfninv/model.py:195: RuntimeWarning: invalid value encountered in double_scalars
  prior_ratio = prior_1 / prior_0


******************************Iteration:6******************************
Move:D, Prior=0.0, RMS=337.00255105470836
Accept: True
******************************Iteration:7******************************
Move:D, Prior=0.0, RMS=337.00255105470836
Accept: True


/Volumes/SD_Card/Thesis_project/pydfninv/model.py:195: RuntimeWarning: invalid value encountered in double_scalars
  prior_ratio = prior_1 / prior_0
/Volumes/SD_Card/Thesis_project/pydfninv/model.py:195: RuntimeWarning: invalid value encountered in double_scalars
  prior_ratio = prior_1 / prior_0


******************************Iteration:8******************************
Move:D, Prior=0.0, RMS=337.00255105470836
Accept: True
******************************Iteration:9******************************
Move:D, Prior=0.0, RMS=337.00255105470836
Accept: True


/Volumes/SD_Card/Thesis_project/pydfninv/model.py:195: RuntimeWarning: invalid value encountered in double_scalars
  prior_ratio = prior_1 / prior_0


In [ ]:
# Initialize MCMC
prior_current = 1 #prior_probability(s_current)

dfninv.run_forward_simulation(s_current)
syn_data_current = dfninv.read_simulation_results()
rms_0, rms_1, ratio_0 = likelihood_ratio(syn_data_current, syn_data_current, field_observation)
status = {'model_id': 0, 'RMS': rms_0, 'fractures': s_current}
dfninv.save_accepted_model(status, model_id=0, save_flag=True)

In [ ]:
initial_dfn = project_path+'/accept_models/model_0/full_mesh.vtk'
camera_angle = [0, 30, 0]
dfnmesh_render = vtk_meshrender(initial_dfn, station_coordinates, camera_ang = camera_angle)
vtk_show(dfnmesh_render, 600, 600)

In [ ]:
# Start loop
max_iteration = 100
i_iter = 0
accept_case = 0
model_trace = np.zeros([n_total, 6, max_iteration])

while i_iter < max_iteration:
    print('{0}Iteration:{1:d}{2}'.format('*' * 20, i_iter, '*' * 20))
    model_trace[:, :, 0] = s_current
    accept_flag = False

    s_proposed = Matrix_normal_distribution(s_current, sigma_f, sigma_v).get_proposal()
    dfninv.run_forward_simulation(s_proposed)
    syn_data_proposed = dfninv.read_simulation_results()
    prior_proposed = 1 #prior_probability(s_proposed)
    print(prior_proposed)
    print(s_proposed)
    rms_current, rms_proposed, ratio_ll = likelihood_ratio(syn_data_current, syn_data_proposed, field_observation)
    ratio_prior = prior_proposed / prior_current
    ratio_proposal = 1

    # calculate acceptance ratio
    alpha_1 = min(1, ratio_ll * ratio_prior * ratio_proposal)
    print(ratio_prior)
    print('Likelihood ratio = {:.2f}'.format(ratio_ll))
    print(alpha_1)
    # print('Acceptance Ratio = {:3f}'.format(alpha_1))

    if alpha_1 > random():
        accept_flag = True
        accept_case += 1

        s_current = s_proposed
        prior_current = prior_proposed
        rms_current = rms_proposed
        print('### Accept! RMS = {} ###'.format(rms_current))

    status = {'model_id': accept_case, 'RMS': rms_current, 'fractures': s_current}
    dfninv.save_accepted_model(status, model_id=accept_case, save_flag=accept_flag)
    i_iter += 1

In [ ]:
# Analyze rms log
with open(project_path + '/mcmc_log.txt', 'r') as logfile:
    mcmc_log = logfile.readlines()
    separator = '*' * 60 + '\n'
    idx_sep = [i for i, j in enumerate(mcmc_log) if j == separator]

    n_fractures = idx_sep[1] - idx_sep[0] - 3
    n_model = len(idx_sep)
    
    rms = []
    for i in idx_sep:
        rms.append(float(mcmc_log[i+2].replace('\n', '')))

In [ ]:
latexify()
fig = plt.figure()
plt.plot(rms)
plt.xlabel('Iteration')
plt.ylabel('RMS')
plt.title('Root-Mean-Square Error Evolution as MCMC Iteration')
plt.savefig(project_path + '/rms_iteration.pdf')
plt.show()

In [ ]:
latexify()
fig = plt.figure()
cx = model_trace[3, 1, :]
plt.plot(cx)
# plt.xlabel('Iteration')
# plt.ylabel('RMS')
# plt.title('Root-Mean-Square Error Evolution as MCMC Iteration')
# plt.savefig(project_path + '/rms_iteration.pdf')
plt.show()

In [ ]:
print(cx.shape)